<img src="https://opea.dev/wp-content/uploads/sites/9/2024/04/opea-horizontal-color.svg" alt="OPEA Logo">

# Deploy and Learn ChatQnA using OPEA on Intel Tiber AI Cloud 
## Replace your LLM model

To deploy the `chatqna` application with a custom Hugging Face model (e.g., `deepseek-ai/DeepSeek-R1-Distill-Qwen-7B`), run the following command:

In [ ]:
!helm upgrade --install chatqna oci://ghcr.io/opea-project/charts/chatqna \
  --set vllm.LLM_MODEL_ID=deepseek-ai/DeepSeek-R1-Distill-Qwen-7B \
  -f cpu-values.yaml && kubectl logs -l app=chatqna --tail=50

## 1. Update config map 

ConfigMaps are used to store configuration settings that your applications need to run. These settings can be environment variables, command-line arguments, or configuration files. By using ConfigMaps, you can update your application's configuration without rebuilding or redeploying your containers.

✅ Benefits of Using ConfigMaps

- Separation of Concerns: Keeps configuration separate from application code.​
- Easy Updates: Allows you to update configuration without redeploying your application.​
- Environment Specific Configurations: Facilitates different configurations for different environments (e.g., development, staging, production).​
- Dynamic Configuration: Enables applications to adapt to configuration changes in real-time.​

To list all ConfigMaps in the current namespace:​​

In [38]:
!kubectl get configmaps

NAME                            DATA   AGE
chatqna-chatqna-ui-config       4      4d22h
chatqna-data-prep-config        20     4d22h
chatqna-nginx-config            7      4d22h
chatqna-retriever-usvc-config   19     4d22h
chatqna-tei-config              11     4d22h
chatqna-teirerank-config        11     4d22h
chatqna-vllm-config             11     4d22h
kube-root-ca.crt                1      25d


This command updates the **LLM_MODEL_ID** from `meta-llama/Meta-Llama-3-8B-Instruct` to `deepseek-ai/DeepSeek-R1-Distill-Qwen-7B`, a multimodal large language model developed by Alibaba Cloud's Qwen team. ​

In [57]:
!kubectl patch configmap chatqna-vllm-config -p '{"data": {"LLM_MODEL_ID": "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"}}'

configmap/chatqna-vllm-config patched (no change)


## 2. Update the POD with the new model

Check the current deployments
Run the following command to list all the deployments in your Kubernetes cluster:

In [46]:
!kubectl get deployments

NAME                      READY   UP-TO-DATE   AVAILABLE   AGE
chatqna                   1/1     1            1           4d22h
chatqna-chatqna-ui        1/1     1            1           4d22h
chatqna-data-prep         1/1     1            1           4d22h
chatqna-nginx             1/1     1            1           4d22h
chatqna-redis-vector-db   1/1     1            1           4d22h
chatqna-retriever-usvc    1/1     1            1           4d22h
chatqna-tei               1/1     1            1           4d22h
chatqna-teirerank         1/1     1            1           4d22h
chatqna-vllm              1/1     1            1           4d22h


This helps you confirm that the deployment (`chatqna-vllm` in this case) exists and is ready to be restarted.

Restart the deployment to apply the new model without needing to manually delete pods. This command triggers a rolling restart, meaning Kubernetes will gracefully recreate the pods one by one, minimizing downtime.

In [47]:
!helm upgrade --install chatqna oci://ghcr.io/opea-project/charts/chatqna --set vllm.LLM_MODEL_ID=deepseek-ai/DeepSeek-R1-Distill-Qwen-7B -f cpu-values.yaml && kubectl logs -l app=chatqna --tail=50

deployment.apps/chatqna-vllm restarted


**Note**:
Restarting the deployment ensures the new model files/configurations are picked up without needing to redeploy the entire application.

## 3. Check the new pods. 

After restarting, list all pods to verify the new pods are running:

In [52]:
!kubectl get pods

NAME                                       READY   STATUS    RESTARTS   AGE
chatqna-868d98c5bf-vb2bk                   1/1     Running   0          4d22h
chatqna-chatqna-ui-ffd74c8d8-j67gr         1/1     Running   0          41m
chatqna-data-prep-59849c8885-xn8pq         1/1     Running   0          4d22h
chatqna-nginx-6c855d856c-4nds9             1/1     Running   0          4d22h
chatqna-redis-vector-db-8566ffdb78-f8d62   1/1     Running   0          4d22h
chatqna-retriever-usvc-57c8c4c7d5-9rfrd    1/1     Running   0          4d22h
chatqna-tei-9c46456c7-z89lc                1/1     Running   0          4d22h
chatqna-teirerank-5d4c49cd8d-sq7jc         1/1     Running   0          4d22h
chatqna-vllm-58d8d9cf69-xtgb8              1/1     Running   0          11m


In [50]:
vllm_pod = !kubectl get pods --no-headers | grep chatqna-vllm | awk '{print $1}'
vllm_pod = vllm_pod[0]

vLLM pod: chatqna-vllm-58d8d9cf69-xtgb8


Look for the new pods related to your deployment (e.g., `chatqna-vllm-xxxx`).

Check environment variables inside the pod. Verify that the correct model environment variable (LLM_MODEL_ID) is set inside the running pod.

In [53]:
# Check you replaced the variable
!kubectl exec -it {vllm_pod} -- printenv LLM_MODEL_ID

Defaulted container "vllm" out of: vllm, model-downloader (init)
meta-llama/Meta-Llama-3-8B-Instruct


Confirm the model was correctly updated
Make sure that the `LLM_MODEL_ID` printed matches the model you intended to deploy.
If it doesn’t, double-check your deployment configuration and environment variable settings.